In [1]:
from __future__ import print_function

import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn import tree

from matplotlib import pyplot
import numpy as np
import random
%matplotlib inline
pd.options.mode.chained_assignment = None
bank = pd.read_csv('bank-full.csv',sep=";")


In [2]:
m = bank.shape[1]
X = bank.iloc[:,0:(m-1)]
y = bank.iloc[:,(m-1):]

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=33)

In [4]:
X_train.dtypes

age           int64
job          object
marital      object
education    object
default      object
balance       int64
housing      object
loan         object
contact      object
day           int64
month        object
duration      int64
campaign      int64
pdays         int64
previous      int64
poutcome     object
dtype: object

In [5]:
X_train_onehot = pd.get_dummies(X_train)
X_test_onehot = pd.get_dummies(X_test)
y_train.loc[:,'y'] = y_train.loc[:,'y'].map({'no':0,'yes':1})
y_test.loc[:,'y'] = y_test.loc[:,'y'].map({'no':0,'yes':1})

In [6]:
X_train_onehot

,age,balance,day,duration,campaign,pdays,previous,job_admin.,job_blue-collar,job_entrepreneur,...,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown
38015,30,303,14,176,2,365,4,0,0,1,...,0,0,1,0,0,0,1,0,0,0
44341,29,0,29,99,6,-1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
24269,49,7,17,50,1,-1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
4680,38,273,20,314,2,-1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
6450,45,-173,27,315,2,-1,0,1,0,0,...,0,0,1,0,0,0,0,0,0,1
15252,27,24,17,320,2,-1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
28981,50,1088,2,749,1,-1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
41779,74,1124,13,200,2,-1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
11037,39,1551,17,264,16,-1,0,0,1,0,...,1,0,0,0,0,0,0,0,0,1
18916,53,0,4,134,2,-1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [17]:
T=50 #Max number of trees

m = X_train_onehot.shape[0]
N = X_train_onehot.shape[1]
feature_portion = 0.5
W = [1.0/m for i in range(0,m,1)]
W = pd.DataFrame({'Weight':list(W)},index = X_train_onehot.index)
feature_record = pd.DataFrame()

# List of trees
clfs = []
# List of weights of trees
alphas = []

for i in range(1,T+1):
    # Prepare training sample subset (bagging)
    X_train_onehot_sample, X_test_onehot_sample, y_train_sample, y_test_sample = train_test_split(X_train_onehot, y_train, test_size=0.20, random_state=33)
    selected_features =  [random.randint(0,N-1) for j in range(0,int(round(N*feature_portion)))]
    feature_record=feature_record.append(pd.DataFrame([selected_features]), ignore_index=True)    
    X_train_onehot_sample = X_train_onehot_sample.iloc[:,selected_features]
    X_test_onehot_sample = X_test_onehot_sample.iloc[:,selected_features]
    # Prepare tree classifier
    clf = tree.DecisionTreeClassifier(criterion='entropy',max_depth=None)
    # Weight of current training sample
    w_ = W.loc[X_train_onehot_sample.index,"Weight"].tolist()
    
    # Train decision tree
    clf.fit(X=X_train_onehot_sample, y=y_train_sample,sample_weight=w_)
    # Make prediction
    pred = clf.predict(X_train_onehot.iloc[:,selected_features])
    
    # Calculate weighted error rate of current tree
    print("sum(np.array(W)): {}".format(sum(np.array(W))))
    eps = sum(np.array(W)[(np.array(pred) != np.array(y_train))[0]]) / sum(np.array(W))
    print("eps: ", eps)
    if eps < 1e-20 :
        print("eps == {}. Break".format(eps))
        break
    # Compute weight of decision tree
    alpha = (1/2)*np.log( (m-1)*(1-eps)/eps )
    print("Alpha:", alpha)
    alphas.append(alpha)
    
    
#   print( np.array([ int(y==p) for y,p in zip(np.ravel(y_train), pred) ])[np.where(W>0)]         )
    print("if all focused preditcted correct then zero:",sum( (np.array(pred) != np.array(y_train))[0][np.ravel(np.array(W)>0) & np.ravel(~np.isnan(W))]) ,sep="")
    
    # Update weight of training sample
    if alpha > 0 :
        exp_alphas = [ np.exp(-alpha) if y==p else np.exp(alpha) for y,p in zip(np.ravel(y_train), pred) ]
        W = np.multiply(W, exp_alphas) / np.sum(np.multiply(W, exp_alphas))

        
        # Save trained tree to list 
        clfs.append(clf)
    # If alpha < 0, reject the tree
        print('in if then loop')
    else :
        print("Tree {} is rejected.".format(i))


        

    
    #X.iloc[X_train.index,:]
    print('The accuracy of Tree ',str(i), ' is ',round(clf.score(X=X_test_onehot_sample, y=y_test_sample),5),sep='')
    


sum(np.array(W)): [1.]
eps:  [0.11890999]
Alpha: [6.21708486]
if all focused preditcted correct then zero:4032
in if then loop
The accuracy of Tree 1 is 0.86007
sum(np.array(W)): [1.]
eps:  [0.09272097]
Alpha: [6.35611606]
if all focused preditcted correct then zero:1185
in if then loop
The accuracy of Tree 2 is 0.88617
sum(np.array(W)): [1.]
eps:  [0.20140627]
Alpha: [5.90445251]
if all focused preditcted correct then zero:3968
in if then loop
The accuracy of Tree 3 is 0.83692
sum(np.array(W)): [1.]
eps:  [0.54252198]
Alpha: [5.13043851]
if all focused preditcted correct then zero:19600
in if then loop
The accuracy of Tree 4 is 0.45842
sum(np.array(W)): [1.]
eps:  [0.15333303]
Alpha: [6.07003599]
if all focused preditcted correct then zero:4456
in if then loop
The accuracy of Tree 5 is 0.80832
sum(np.array(W)): [1.]
eps:  [0.33606269]
Alpha: [5.55613339]
if all focused preditcted correct then zero:4058
in if then loop
The accuracy of Tree 6 is 0.84916
sum(np.array(W)): [1.]
eps:  [0.1

In [8]:
clf2[0] = tree.DecisionTreeClassifier(criterion='entropy',max_depth=None)

NameError: name 'clf2' is not defined

In [ ]:
feature_record

In [18]:
alphas /= sum(alphas)


ensemble_prob = np.zeros(len(X_train_onehot))
for i,clf in enumerate(clfs) :
    prob = clf.predict_proba(X_train_onehot.iloc[:,list(feature_record.iloc[i,:])])
    ensemble_prob += prob[:,1]*alphas[i]
    #print(ensemble_prob)

#ensemble_prob /= len(clfs)
#ensemble_pred = ensemble_prob
#ensemble_pred[ensemble_pred>=0.5]=1
#ensemble_pred[ensemble_pred<0.5]=0
#print(sum(ensemble_pred))
#print(ensemble_prob)
#ensemble_pred = [ int(pb>0.5) for pb in ensemble_prob ]
#print(len(ensemble_pred))
ensemble_pred= ensemble_prob
ensemble_pred[ensemble_prob>=0.5]=1
ensemble_pred[ensemble_prob<0.5]=0
sum(ensemble_pred)

3425.0

In [19]:
np.mean((np.array(y_train)==np.array(ensemble_pred))[0])

0.8989913884628996

In [20]:
alphas /= sum(alphas)


ensemble_prob = np.zeros(len(X_test_onehot))
for i,clf in enumerate(clfs) :
    prob = clf.predict_proba(X_test_onehot.iloc[:,list(feature_record.iloc[i,:])])
    ensemble_prob += prob[:,1]*alphas[i]
    #print(ensemble_prob)

#ensemble_prob /= len(clfs)
#ensemble_pred = ensemble_prob
#ensemble_pred[ensemble_pred>=0.5]=1
#ensemble_pred[ensemble_pred<0.5]=0
#print(sum(ensemble_pred))
#print(ensemble_prob)
#ensemble_pred = [ int(pb>0.5) for pb in ensemble_prob ]
#print(len(ensemble_pred))
ensemble_pred= ensemble_prob
ensemble_pred[ensemble_prob>=0.5]=1
ensemble_pred[ensemble_prob<0.5]=0
sum(ensemble_pred)

357.0

In [48]:
test1=((np.ravel(y_test)==np.ravel(ensemble_pred)))
np.mean(test1)

0.8926833584004247

array([ True,  True,  True, ...,  True,  True,  True])

In [29]:
pred = np.array(ensemble_pred)
corr = [ int(y==p) for y,p in zip(pred, np.array(y_test)) ]
print( np.mean(corr) )

0.8926833584004247


In [ ]:
feature_record

In [22]:
importances = clf.feature_importances_ 
importance_frame = pd.DataFrame({'Importance': list(importances), 'Feature': list(X_train_onehot.columns)})
importance_frame.sort_values(by = 'Importance', inplace = True)
importance_frame.plot(kind = 'barh', x = 'Feature', figsize = (16,100), color = 'orange')

ValueError: arrays must all be same length

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [23]:
from sklearn.ensemble import RandomForestClassifier

In [27]:
rfc = RandomForestClassifier()
rfc.fit(X_train_onehot, y_train)
print('The accuracy of Random Forest Classifier on testing set:', rfc.score(X_test_onehot, y_test))
np.mean(np.array(rfc.predict(X_test_onehot))==np.array(y_test))

/anaconda3/envs/face/lib/python3.6/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


The accuracy of Random Forest Classifier on testing set: 0.8979916836238166


0.8344194913361421

In [28]:
pred = np.array(rfc.predict(X_test_onehot))
corr = [ int(y==p) for y,p in zip(pred, np.array(y_test)) ]
print( np.mean(corr) )

0.8979916836238166


In [ ]:
import xgboost as xgb
from xgboost import XGBClassifier
from xgboost import DMatrix
xgbc = XGBClassifier()
xgbc.fit(X_train_onehot, y_train)
print('The accuracy of eXtreme Gradient Boosting Classifier on testing set:', xgbc.score(X_test_onehot, y_test))

xgdmat = DMatrix(X_train_onehot, y_train) 

In [ ]:
from sklearn.model_selection import GridSearchCV
cv_params = {'max_depth': [3,5,7], 'min_child_weight': [1,3,5]}
ind_params = {'learning_rate': 0.1, 'n_estimators': 1000, 'seed':0, 'subsample': 0.8, 'colsample_bytree': 0.8, 
             'objective': 'binary:logistic'}
optimized_GBM = GridSearchCV(xgb.XGBClassifier(**ind_params), 
                            cv_params, 
                             scoring = 'accuracy', cv = 5, n_jobs = -1) 
# Optimize for accuracy since that is the metric used in the Adult Data Set notation

optimized_GBM.fit(X_train_onehot, y_train)

In [ ]:
optimized_GBM.best_estimator_

In [ ]:
optimized_GBM2 = GridSearchCV(cv=5, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=3, missing=None, n_estimators=1000, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'subsample': [0.7, 0.8, 0.9], 'learning_rate': [0.1, 0.01]},
       pre_dispatch='2*n_jobs', refit=True, scoring='accuracy', verbose=0)

optimized_GBM2.fit(X_train_onehot, y_train)

In [ ]:
optimized_GBM2.best_estimator_

In [ ]:
our_params = {'eta': 0.1, 'seed':0, 'subsample': 0.8, 'colsample_bytree': 0.8, 
             'objective': 'binary:logistic', 'max_depth':3, 'min_child_weight':3} 
# Grid Search CV optimized settings

cv_xgb = xgb.cv(params = our_params, dtrain = xgdmat, num_boost_round = 3000, nfold = 5,
                metrics = ['error'], # Make sure you enter metrics inside a list or you may encounter issues!
                early_stopping_rounds = 100) # Look for early stopping that minimizes error

cv_xgb.tail(5)

In [ ]:
from xgboost import plot_tree
from matplotlib import pyplot
%matplotlib inline
import seaborn as sns
sns.set(font_scale = 1.5)

plot_tree(xgbc,rankdir='LR')
fig = pyplot.gcf()
fig.set_size_inches(150, 100)
#fig.savefig('tree.png')

In [ ]:
from xgboost import plot_importance
plot_importance(xgbc)
fig = pyplot.gcf()
fig.set_size_inches(150, 100)

In [ ]:
importances = xgbc.feature_importances_
importance_frame = pd.DataFrame({'Importance': list(importances), 'Feature': list(X_train_onehot.columns)})
importance_frame.sort_values(by = 'Importance', inplace = True)

In [ ]:
importance_frame.plot(kind = 'barh', x = 'Feature', figsize = (16,100), color = 'orange')

In [ ]:
fig.savefig("importance.png", format="PNG")